In [2]:
# Using an LLM to do website summatization - basic use case
!pip install firecrawl-py -q
!pip install requests -q
!pip install ipython -q
!pip install openai -q

In [4]:
# IMPORTS
import os
import requests
from IPython.display import Markdown, display
from openai import OpenAI
from google.colab import userdata
from firecrawl import FirecrawlApp

In [20]:
class Summarizer:
  def __init__(self, url, openai_key, firecrawl_key):
    self.url = url
    self.openai_key = openai_key
    self.firecrawl_key = firecrawl_key

  def check_apis(self):
    if not self.openai_key:
      raise ValueError("OpenAI key is missing")
    if not self.firecrawl_key:
      raise ValueError("Firecrawl key is missing")
    if not self.openai_key.starswith("sk-proj-"):
      print("An API key was found, but it doesn't start sk-proj-; please check you open ai key")
    if not self.firecrawl_key.starswith("fc-"):
      print("An API key was found, but it doesn't start fc-; please check you firecrawl key")

  def scrape(self):
    app = FirecrawlApp(api_key=self.firecrawl_key)
    page = app.scrape_url(self.url, params={'formats': ['markdown', 'html']})
    self.page_mk = page["markdown"]
    return self.page_mk

  def prompts(self):
    self.scrape()
    self.system_prompt =  "You are an assistant that analyzes the contents of a website \
    and provides a short summary, ignoring text that might be navigation related. \
    Respond in markdown."

    self.user_prompt = "\nThe contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"

    self.user_prompt += self.page_mk

    return self.system_prompt, self.user_prompt

  def messages_for(self):
    self.prompts()
    return [{
        "role": "system",
        "content": self.system_prompt
    }, {
        "role": "user",
        "content": self.user_prompt
    }]

  def summarize(self):
    openai = OpenAI(api_key=self.openai_key)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = self.messages_for()
    )
    return display(Markdown(response.choices[0].message.content))

In [9]:
# API KEYS:
OPEN_AI_KEY = userdata.get('OPEN_AI_KEY')
FIRECRAWL_KEY = userdata.get('FIRECRAWL_API_KEY')

In [22]:
sum = Summarizer("https://github.com/ronaldborja", OPEN_AI_KEY, FIRECRAWL_KEY)
sum.summarize()

# Summary of the Website

The website is a GitHub profile belonging to Ronald Borja Román, a student of Chemical Engineering and Software Engineering from Colombia. He expresses an interest in the intersection of nature and AI, as well as various tech domains including web development, machine learning, optimization, bioprocesses, bioinformatics, cloud computing, and IoT.

## Achievements and Interests
- Ronald has received multiple GitHub achievements, indicating active participation in collaborative software projects.
- He is open to networking as demonstrated by links to his LinkedIn and other social media accounts.

## Projects
Ronald has several pinned repositories showcasing his projects:
1. **chemsolid-app**: A desktop application that assists in sizing equipment for the solids industry.
2. **house_price_prediction_api**: A full-stack application for predicting house prices using machine learning.
3. **banking-system**: Development of a banking system utilizing various web technologies.
4. **decision-tree-implementation**: A project focused on implementing decision trees from scratch.
5. **app-inventory**: Software development for inventory management.
6. **knn-implementation**: Implementation of a K-Nearest Neighbors model using numpy and pandas.

## Contact Information
Ronald can be reached via email at [rborjaroman@gmail.com](mailto:rborjaroman@gmail.com). 

This profile portrays a budding engineer actively engaging in community coding and open-source projects while exploring his interests in technology and engineering.